# Temporal Trend Analysis of Urban Morphology Sustainability  
## Mecklenburg County (1990–2023)

### Research Question
Has urban morphology in Mecklenburg County exhibited a trend toward greater
sustainability over time?

### Operational Definition of “Trend”
A temporal trend is defined as a **persistent, statistically identifiable
directional change** in an annual urban morphology indicator over the study
period.

Trend is evaluated using:
1. Long-term smoothing and signal extraction  
2. Formal monotonic and parametric trend tests  
3. Structural break analysis to detect regime changes  
4. Distributional and density-based diagnostics


In [ ]:
# ============================================================
# Tier 0 — Setup
# ============================================================
import warnings
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.io as pio
from rpy2.robjects.lib.dplyr import summarize

from statsmodels.nonparametric.smoothers_lowess import lowess
from statsmodels.tsa.stattools import adfuller, kpss
import statsmodels.api as sm
from scipy.stats import gaussian_kde
import ruptures as rpt
from pymannkendall import original_test as mk_test

warnings.filterwarnings("ignore")
pd.set_option("display.float_format", "{:.3f}".format)
pio.renderers.default = "notebook_connected"


In [ ]:
# ============================================================
# Tier 0 — Load & Filter Data
# ============================================================
ABT = gpd.read_file(
    "../../../../Data/Final_dataset/ABT/ABT.gpkg",
    layer="subdivisions"
)

ABT = ABT[(ABT["year"] >= 1990) & (ABT["year"] <= 2023)]

In [ ]:
measure = "COMPACTNESS_SUM"

# ============================================================
# Tier 1 — Annual Distributional Summary Series
# ============================================================
This tier constructs annual time series capturing **central tendency**
(mean, median) and **dispersion** (standard deviation, interquartile range).


In [ ]:
annual_stats = (
    ABT.groupby("year")[measure]
    .agg(
        mean="mean",
        median="median",
        std="std",
        iqr=lambda x: x.quantile(0.75) - x.quantile(0.25)
    )
    .reset_index()
)

years = annual_stats["year"].values
annual_stats

# ============================================================
# Tier 2 — Generalized Temporal Trend Diagnostics
# ============================================================
The following function applies **identical temporal diagnostics** to any
annual series, ensuring symmetry across mean, median, and dispersion measures.


In [ ]:
def run_trend_diagnostics(years, series, label, lowess_frac=0.25):
    # -----------------------------
    # Stationarity diagnostics
    # -----------------------------
    adf_p = adfuller(series)[1]
    kpss_p = kpss(series, nlags="auto")[1]

    print(f"\n{label} — Stationarity")
    print(f"ADF p-value  : {adf_p:.4f}")
    print(f"KPSS p-value : {kpss_p:.4f}")

    # -----------------------------
    # LOWESS smoothing
    # -----------------------------
    trend = lowess(series, years, frac=lowess_frac)[:, 1]

    # -----------------------------
    # Visual trend
    # -----------------------------
    plt.figure(figsize=(14, 5))
    plt.plot(years, series, alpha=0.4, label="Observed")
    plt.plot(years, trend, linewidth=3, label="LOWESS")
    plt.title(f"{label}: Long-Term Trend")
    plt.xlabel("Year")
    plt.ylabel(label)
    plt.legend()
    plt.grid(alpha=0.3)
    plt.show()

    # -----------------------------
    # Derivatives
    # -----------------------------
    slope = np.gradient(trend, years)
    acceleration = np.gradient(slope, years)

    # -----------------------------
    # Mann–Kendall
    # -----------------------------
    mk = mk_test(series)
    print(f"{label} — Mann–Kendall")
    print(f"Trend: {mk.trend}")
    print(f"Sen slope: {mk.slope:.4f}")
    print(f"p-value: {mk.p:.4f}")

    # -----------------------------
    # HAC linear trend
    # -----------------------------
    X = sm.add_constant(years)
    model = sm.OLS(series, X).fit(
        cov_type="HAC",
        cov_kwds={"maxlags": 2}
    )
    print(model.summary())

    # -----------------------------
    # Structural breaks
    # -----------------------------
    y_std = (series - np.mean(series)) / np.std(series)
    algo = rpt.Pelt(model="rbf", min_size=5).fit(y_std)
    breaks = algo.predict(pen=np.log(len(series)))
    break_years = [years[i - 1] for i in breaks[:-1]]

    print(f"{label} — Structural breaks:", break_years)

    plt.figure(figsize=(14, 5))
    plt.plot(years, series, marker="o")
    for yr in break_years:
        plt.axvline(yr, linestyle="--", color="red", alpha=0.6)
    plt.title(f"{label}: Structural Breaks")
    plt.xlabel("Year")
    plt.ylabel(label)
    plt.grid(alpha=0.3)
    plt.show()

### Applying Identical Diagnostics Across Summary Measures


In [ ]:
for col in ["mean", "median", "std", "iqr"]:
    run_trend_diagnostics(
        years=years,
        series=annual_stats[col].values,
        label=f"{measure.upper()} — {col.capitalize()}"
    )


# ============================================================
# Tier 3 — Distributional Shape Dynamics (Density–Value–Year)
# ============================================================
This tier examines **full distributional evolution**, including dispersion,
polarization, and potential bimodality not captured by summary statistics.


In [ ]:
# Build density matrix
value_grid = np.linspace(ABT[measure].min(), ABT[measure].max(), 200)
years_sorted = np.sort(ABT["year"].unique())
Z = np.zeros((len(years_sorted), len(value_grid)))

for i, yr in enumerate(years_sorted):
    vals = ABT.loc[ABT["year"] == yr, measure].dropna().values
    if len(vals) < 5 or np.std(vals) == 0:
        Z[i, :] = np.nan
    else:
        kde = gaussian_kde(vals)
        Z[i, :] = kde(value_grid)


In [ ]:
# 3D Density Surface (Plotly)
X, Y = np.meshgrid(value_grid, years_sorted)

fig = go.Figure(
    data=[go.Surface(x=X, y=Y, z=Z)]
)

fig.update_layout(
    title=f"3D Density Surface: {measure} Distribution Over Time",
    width=1400,
    height=900,
    scene=dict(
        xaxis_title=f"{measure} value",
        yaxis_title="Year",
        zaxis_title="Density",
        aspectmode="manual",
        aspectratio=dict(x=1.5, y=2.5, z=0.8)
    ),
    scene_camera=dict(eye=dict(x=1.6, y=1.8, z=0.9))
)

fig.show()


In [ ]:
import numpy as np
import plotly.graph_objects as go

# ============================================================
# Prepare years
# ============================================================
years_sorted = np.sort(ABT["year"].unique())

# ============================================================
# Common binning across all years
# ============================================================
n_bins = 40
bin_edges = np.linspace(
    ABT[measure].min(),
    ABT[measure].max(),
    n_bins + 1
)
bin_centers = 0.5 * (bin_edges[:-1] + bin_edges[1:])

# ============================================================
# Build figure
# ============================================================
fig = go.Figure()

for yr in years_sorted:
    vals = ABT.loc[ABT["year"] == yr, measure].dropna().values
    if len(vals) < 10:
        continue

    counts, _ = np.histogram(vals, bins=bin_edges)

    # Normalize within year (distributional shape only)
    if counts.max() > 0:
        counts = counts / counts.max()

    fig.add_trace(
        go.Scatter(
            x=bin_centers,
            y=counts + yr,                 # vertical stacking by year
            mode="lines+markers",

            line=dict(color="darkred", width=1),

            # IMPORTANT: almost-invisible markers (not zero opacity!)
            marker=dict(
                size=6,
                opacity=0.001,
                color="black"
            ),

            opacity=0.7,
            showlegend=False,

            text=[yr] * len(bin_centers),  # year for hover
            customdata=counts,             # normalized frequency for hover

            hoveron="points",

            hovertemplate=(
                "Year: %{text}<br>"
                + f"{measure}: "
                + "%{x:.3f}<br>"
                + "Frequency (normalized): %{customdata:.3f}"
            )
        )
    )

# ============================================================
# Layout: grids, axes, interaction
# ============================================================
fig.update_layout(
    title=f"Stacked Annual Histograms of {measure} (1980–2023)",
    xaxis_title=f"{measure} value",
    yaxis_title="Year",
    height=1200,
    width=1700,
    template="simple_white",

    hovermode="closest",

    # X axis (BAD)
    xaxis=dict(
        showgrid=True,
        gridcolor="rgba(0,0,0,0.25)",
        gridwidth=1,
        zeroline=False,
        nticks=10,
        minor=dict(
            ticks="inside",
            showgrid=True,
            gridcolor="rgba(0,0,0,0.10)",
            gridwidth=0.5
        )
    ),

    # Y axis (Year)
    yaxis=dict(
        showgrid=True,
        gridcolor="rgba(0,0,0,0.25)",
        gridwidth=1,
        zeroline=False,
        dtick=5,  # major grid every 5 years
        minor=dict(
            ticks="inside",
            showgrid=True,
            gridcolor="rgba(0,0,0,0.10)",
            gridwidth=0.5,
            dtick=1
        )
    )
)

fig.show()


In [ ]:
!jupyter nbconvert --to html --no-input time_series_univariate.ipynb --output ../../../../output/Notebook_Outputs/time_series/time_series_EDA1th.html